In [11]:
import os
import sys

In [12]:
if 'Carbonate' in os.getcwd().split('/'):
    sys.path.insert(0,'/N/slate/hhansar/hgenv/lib/python3.10/site-packages')

In [13]:
import transformers

In [14]:
from transformers import AutoImageProcessor, VideoMAEForPreTraining

In [15]:
import numpy as np
import torch

In [16]:
from PIL import Image

In [17]:
from pathlib import Path

def get_fpathlist(subjdir, ds_rate=1):
    # read the image file names into fpathlist
    # subjdir = ['008MS'] #@@@
    # vid_root = r"/N/project/infant_image_statistics/03_JPG_AllMasters/"
    
    fpathlist = sorted(list(Path(subjdir).iterdir()), 
                       key=lambda x: x.name)
    fpathlist = [str(fpath) for fpath in fpathlist if fpath.suffix=='.jpg']
    fpathlist = fpathlist[::ds_rate]
    return fpathlist

pathlist = get_fpathlist(r"/N/project/infant_image_statistics/03_JPG_AllMasters/003MT")
paths = pathlist[:16]

In [18]:
import os
from PIL import Image
import numpy as np

images = []
for path in paths:
    with Image.open(path) as img:
        # add an axis to the image array to (1, num_channels, height, width)
        img_array = np.expand_dims(np.array(img), axis=0).transpose(0, 3, 1, 2)
        images.append(img_array)

# concatenate the images into a numpy array
concatenated_images = np.concatenate(images, axis=0)

In [19]:
num_frames = 16
video = list(concatenated_images)

In [20]:
image_processor = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base")
model = VideoMAEForPreTraining.from_pretrained("MCG-NJU/videomae-base")

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [21]:
pixel_values = image_processor(video, return_tensors="pt").pixel_values
pixel_values.shape

/N/u/hhansar/Carbonate/.local/lib/python3.10/site-packages/transformers/feature_extraction_utils.py:147: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:210.)
  return torch.tensor(value)


torch.Size([1, 16, 3, 224, 224])

In [22]:
num_patches_per_frame = (model.config.image_size // model.config.patch_size) ** 2
seq_length = (num_frames // model.config.tubelet_size) * num_patches_per_frame
bool_masked_pos = torch.randint(0, 2, (1, seq_length)).bool()

In [23]:
print(model.config.patch_size)

16


In [24]:
print(model.config.tubelet_size)

2


In [25]:
print(seq_length)

1568


In [26]:
print(bool_masked_pos.sum())

tensor(789)


In [27]:
outputs = model(pixel_values, bool_masked_pos=bool_masked_pos)
loss = outputs.loss

In [28]:
print(outputs.logits.shape)

torch.Size([1, 789, 1536])


In [29]:
print(loss)

tensor(0.3439, grad_fn=<MseLossBackward0>)
